# Genome Reconstruction from Sequenced Reads (Global Alignment: Needleman-Wunsch)

This notebook demonstrates a simple pipeline for reconstructing a reference genome from a set of sequenced reads using **seeding, hashing, and global alignment (Needleman-Wunsch)**. The workflow includes:

- Defining a reference genome and simulated reads
- Efficient seeding and hashing to find candidate match regions
- Global alignment using the Needleman-Wunsch algorithm
- Consensus-based reconstruction of the reference genome

This version uses **global alignment** (Needleman-Wunsch), which aligns each read to the reference segment end-to-end, penalizing gaps at the ends as well as in the middle.

## 1. Inputs

Define the reference genome and the set of sequenced reads. You can switch between different sets of reads to test the pipeline's robustness.

In [ ]:
# Reference genome (example)
reference_genome = "AGTCGATGCTAGCTTACGGTACCGTAGGCTAGGATCGTACGCTAGGTAGCTAGCTAGCATGCTAGCTAGTTCGATCGTACGTAGCTTAGCTAGCATCGTAGCTAGCTAGGTACGATCGATCGTAGCATGCTAGCTAGGTAGCTAGCTTACGTACGTAGGCTAGCTAGCGTACGATCGTACGCTAGCTAGGCTAGCTAGCTAGCGTACGTAG"

# Example: reads exactly matching the reference genome
sequenced_reads = [
    "AGTCGATGCTAGCTTACGGT", "GATGCTAGCTTACGGTACCG", "CTAGCTTACGGTACCGTAGG",
    "TTACGGTACCGTAGGCTAGG", "TACCGTAGGCTAGGATCGTA", "TAGGCTAGGATCGTACGCTA",
    "AGGATCGTACGCTAGGTAGC", "GTACGCTAGGTAGCTAGCTA", "TAGGTAGCTAGCTAGCATGC",
    "GTAGCTAGCTAGCATGCTAG", "CTAGCTAGTTCGATCGTACG", "AGTTCGATCGTACGTAGCTT",
    "CGATCGTACGTAGCTTAGCT", "GTACGTAGCTTAGCTAGCAT", "TAGCTTAGCTAGCATCGTAG",
    "GCTAGCATCGTAGCTAGCTA", "ATCGTAGCTAGCTAGGTACG", "AGCTAGCTAGGTACGATCGA",
    "CTAGGTACGATCGATCGTAG", "GTACGATCGATCGTAGCATG", "ATCGATCGTAGCATGCTAGC",
    "CGTAGCATGCTAGCTAGGTA", "CATGCTAGCTAGGTAGCTAG", "GCTAGCTAGGTAGCTAGCTT",
    "GTAGGTAGCTAGCTTACGTA", "TAGCTTACGTACGTAGGCTA", "TACGTACGTAGGCTAGCTAG",
    "GTAGGCTAGCTAGCGTACGA", "CTAGCGTACGATCGTACGCT", "CGTACGATCGTACGCTAGCT",
    "TCGATCGTACGCTAGCTAGG", "ACGCTAGCTAGGCTAGCTAG", "TAGGCTAGCTAGCTAGCGTA",
    "AGCTAGCTAGCGTACGTAGT", "TAGCGTACGTAGTCCGATGG", "CGTACGTAGTCCGATGGCAA",
    "AGTCCGATGGCAAGTCTTGA"
]

# To test with noisy or unrelated reads, uncomment one of the following blocks:

#Sequenced reads from a genome mostly similar to reference genome
# sequenced_reads = [
#     "AGTCGATGGT", "GATGCTAACTGACGG", "CTAGCTGACCATACCGCAGG",
#     "TTACGGTATCGTAGGATAGG", "TACCGTAGACGATCCTA", "TAGGCTTGGATCATCCGCTA",
#     "AGGATCGGATGTTAGGTACC", "GTACGCTACTTGGTAGCATA", "GGGGGGGGGGGGGAGGGGGG",
#     "GTAGCTTGATAGCATGTTAG", "TTTGTTTCTTCTTCTTGTTT", "AGTTCGACCATCTC",
#     "AAAAAAAAAAAAAAAAAAAA", "ATTGCGGTACTGGCATTGGG", "TAGCTTAACATCGTGG",
#     "ATGGCGATTGCGATGGCAAA", "ATCGTAGCTGGCTAACC", "AGCCATTGGCATGCCATTGA",
#     "CTAGGTACGATCCGTAGTGG", "AAAAAAAAAAATAAAAAAAA", "ATCGATCGTACCCTAGC",
#     "CGTAGCATGATAGCTTGGTA", "CATGCTTGCTAGGTAACTAG", "GCTAGCTAGGATGCTAGATT",
#     "GGGGGGGGGGGGGAGGGGGG", "AGTAACGTACAAGTCA", "TACGTACGTAGCCTAGTTAG",
#     "GTAGGCTAGATAGCGTTCGA", "CTAGCGTACCATCGTATGCT", "CGTACGATCGTGCACT",
#     "TCGATCGTAGCCTAGCTTGG", "ACGCTAGCTTGGCTAGATAG", "TAGGCTAGCTTCCAGCGTCA",
#     "AGCTAGCTAGCGTACGTGGT", "TAGCGTACGTAGTCCGGTGA", "GGGGGGGGGGGTGGGGGGGG",
#     "GGGGGGGGGGGGGAGGGGGG"
# ]

#Sequenced reads from a genome almostly entirely different from reference genome
# sequenced_reads = [
#     "TTCAGGTCACATGGGTTTCA", "GGAATCCTACGTTCGGAAGT", "CTTTCAGGAGATCCATGTGC",
#     "AACCTGGATGACCAGTTCAA", "GGTTCAAGGACCTTCTTGAC", "ACTGGTGGTACCTTGGAAGA",
#     "CCTAGGTTCACGGATCTGGT", "TCACAGTTTGGGACTCCGTT", "GGATCAGTACCAGCTGTTCA",
#     "CCGTTAGGTTTGAGGTCTTC", "AGTGGATTCAGGATGGTGTT", "TCTGGAGGTTCCATGTAGAG",
#     "GGAACCTTGGAAGTCCATAC", "TAGGTCAGTGGTACCGTGAT", "CTTGATGGTGACCAGGATAC",
#     "TGGAGTTCAGGTGTACCATG", "GAACCTTGACTGGCTTGTGC", "ATCCAGGAGTTCACGGTGAT",
#     "TTGGCAGTACCGTGGGAACT", "GCATCTTACCGATGGCAGGT", "TACCTGGAGTTCAAGGCTGA",
#     "AGGATCTTGGTACCAGTTTC", "GTTACCGGTCATGTAGGCAC", "TCAGGTTGGATCTGGTACAG",
#     "GAGTCTTTCAGGACCGTCAA", "TTAGTCACGGTTAGCATCCA", "CTGGTGAGTCCGGATACCGT",
#     "AACGTGGCTTCAGATCGGTT", "CGGTGACTTGAAGATGCCAG", "TTCAGCTAGGTACCTAGTCA",
#     "ACGTGTCAGGTCTAGGATCC", "GAAGTACCGTTGTGGAGTCA", "TCTGGTTGAGATGCCATACC",
#     "GTTGACCTTGTCAGTGGGAA", "ATCCGTTCAGGATCGTGACC", "CAGTGTTGGAACCTTGTCCA"
# ]

## 2. Utility Functions

Includes hash functions for efficient seeding, scoring matrices, and the Needleman-Wunsch global alignment implementation.

In [ ]:
import numpy as np
from collections import Counter

# DNA base to number mapping for hashing
mapping = {'A': 0, 'T': 1, 'C': 2, 'G': 3}

def custom_hash(seed, base=4):
    """Full hash calculation for a k-mer (seed)"""
    h = 0
    for char in seed:
        h = h * base + mapping[char]
    return h

def rolling_hash(prev_hash, out_char, in_char, k, base=4):
    """Efficiently update hash when sliding window moves by one base"""
    out_val = mapping[out_char]
    in_val = mapping[in_char]
    prev_hash -= out_val * (base ** (k - 1))
    prev_hash = prev_hash * base + in_val
    return prev_hash

# Scoring and penalty values for alignment
match_reward = 2
mismatch_penalty = -1
gap_penalty = -2  # Typically, gap penalty is more negative than mismatch

# Base to index mapping for scoring matrix
base_index = {'A': 0, 'T': 1, 'G': 2, 'C': 3}
scoring_matrix = np.array([
    [match_reward, mismatch_penalty, mismatch_penalty, mismatch_penalty],
    [mismatch_penalty, match_reward, mismatch_penalty, mismatch_penalty],
    [mismatch_penalty, mismatch_penalty, match_reward, mismatch_penalty],
    [mismatch_penalty, mismatch_penalty, mismatch_penalty, match_reward]
])

def get_score(b1, b2):
    """Get score for a pair of bases from the scoring matrix"""
    return scoring_matrix[base_index[b1], base_index[b2]]

def needleman_wunsch(sequence_1, sequence_2):
    """Space Optimsied Global alignment using Needleman-Wunsch algorithm with traceback support."""
    m, n = len(sequence_1), len(sequence_2)
    prev_row = np.zeros(len(sequence_2)+1)
    curr_row = np.zeros(len(sequence_2)+1)
    max_score = 0
    max_pos = (0, 0)
    traceback_matrix = np.zeros((len(sequence_1)+1, len(sequence_2)+1), dtype=int)

    # Initialize first row
    for j in range(n + 1):
        prev_row[j] = j * gap_penalty

    for i in range(1, m + 1):
        curr_row[0] = i * gap_penalty
        for j in range(1, n + 1):
            diag = prev_row[j - 1] + get_score(sequence_1[i-1], sequence_2[j-1])
            up = prev_row[j] + gap_penalty
            left = curr_row[j - 1] + gap_penalty
            curr_row[j] = max(diag, up, left)

            # Traceback matrix: 1 = diag, 2 = up, 3 = left
            if curr_row[j] == diag:
                traceback_matrix[i][j] = 1
            elif curr_row[j] == up:
                traceback_matrix[i][j] = 2
            else:
                traceback_matrix[i][j] = 3

        prev_row, curr_row = curr_row, np.zeros(n + 1)

    # Traceback
    aligned1, aligned2 = "", ""
    i, j = m, n
    while i > 0 or j > 0:
        if i > 0 and j > 0 and traceback_matrix[i][j] == 1:
            aligned_1 = sequence_1[i-1] + aligned_1
            aligned_2 = sequence_2[j-1] + aligned_2
            i -= 1
            j -= 1
        elif i > 0 and (j == 0 or traceback_matrix[i][j] == 2):
            aligned_1 = sequence_1[i-1] + aligned_1
            aligned_2 = '-' + aligned_2
            i -= 1
        else:
            aligned_1 = '-' + aligned_1
            aligned_2 = sequence_2[j-1] + aligned_2
            j -= 1

    return aligned1, aligned2

### Rolling Hash and Utility Function Complexity

**Rolling Hash Table Construction:**
- **Time Complexity:** O(N), where N is the length of the reference genome. Each k-mer hash is computed in O(1) time after the first, thanks to the rolling hash.
- **Space Complexity:** O(N), for storing all k-mer hashes and their positions.

**Custom Hash:**
- **Time Complexity:** O(k), where k is the seed length (very small, typically 3-10).
- **Space Complexity:** O(1).

**Smith-Waterman Local Alignment:**
- **Time Complexity:** O(M×K) per alignment, where M is the read length and K is the reference segment length (usually ≈ M).
- **Space Complexity:** 
                        O(M+K) per alignment without traceback (space-optimized variant).
                        O(M×K) with traceback (as implemented)


## 3. Seeding: Find Matching Regions

Use k-mer seeding and hashing to quickly find candidate regions in the reference genome for each read. The reference genome is indexed using rolling hash values for all k-mers.

In [ ]:
def find_matching_regions(reference_genome, sequenced_reads, seed_length=3):
    """Seeding using first k-mer of each read to find candidate match locations in the reference genome."""
    seed_table = {}
    ref_hash = custom_hash(reference_genome[:seed_length])
    seed_table.setdefault(ref_hash, []).append(0)
    for i in range(1, len(reference_genome) - seed_length + 1):
        ref_hash = rolling_hash(ref_hash, reference_genome[i - 1], reference_genome[i + seed_length - 1], seed_length)
        seed_table.setdefault(ref_hash, []).append(i)
    matching_regions = {}
    for read in sequenced_reads:
        read_seed = read[:seed_length]
        h = custom_hash(read_seed)
        matching_regions[read] = seed_table.get(h, [])
    return matching_regions

### Seeding Complexity

- **Rolling Hash Table Construction:**
  - **Time Complexity:** O(N), where N is the reference genome length.
  - **Space Complexity:** O(N), for storing all k-mer hashes and their positions.

- **Read Seeding Lookup:**
  - **Time Complexity:** O(1) per read for hash lookup (plus O(1) for hash computation).
  - **Space Complexity:** O(R), where R is the number of reads (for storing match lists).

## 4. Global Alignment and Read Placement

For each read, use Needleman-Wunsch to align it to all candidate regions, and record the best alignment and its position.

In [ ]:
# Step 1: Find probable match locations for all the reads
matching_regions = find_matching_regions(reference_genome, sequenced_reads)

# Step 2: Initialize mapping for aligned bases to each reference genome position
ref_alignments = {i: [] for i in range(len(reference_genome))}

# Step 3: For each read, align to candidate regions and record best alignment
for read, positions in matching_regions.items():
    if not positions:
        print(f"Read: {read} → Most probable region: None")
        continue
    best_score = -float('inf')
    best_alignment = ("", "")
    best_position = -1
    for pos in positions:
        ref_segment = reference_genome[pos:pos + len(read)]
        aligned_read, aligned_ref = needleman_wunsch(read, ref_segment)
        # Calculate alignment score
        score = 0
        for r, g in zip(aligned_read, aligned_ref):
            if r == '-' or g == '-':
                score += gap_penalty
            else:
                score += get_score(r, g)
        if score > best_score:
            best_score = score
            best_alignment = (aligned_read, aligned_ref)
            best_position = pos
    # Print alignment summary
    if best_position == -1:
        print(f"Read: {read} → Most probable region: None")
    else:
        print(f"\nRead: {read}")
        print(f"→ Most probable region (starting at {best_position}): {reference_genome[best_position:best_position+len(read)]}")
        print("→ Global Alignment (Needleman-Wunsch):")
        print(f"   Read : {best_alignment[0]}")
        print(f"   Ref  : {best_alignment[1]}")
        # Record aligned bases for consensus
        ref_pos = best_position
        ref_index = 0
        for r_base, g_base in zip(*best_alignment):
            if g_base != '-':
                ref_alignments[ref_pos + ref_index].append(r_base)
                ref_index += 1

### Alignment Step Complexity

For each read, we align it to all candidate regions using Smith-Waterman:

- **Time Complexity:** O(R × C × M × K)
  - R = number of reads
  - C = average number of candidate regions per read
  - M = read length
  - K = reference segment length (usually ≈ M)
- **Space Complexity:** O(M × K) if traceback is used to reconstruct aligned sequences (which has been implemented).

O(M + K) only if traceback is avoided and only the alignment score is required (not the aligned sequences).


## 5. Consensus Genome Reconstruction

For each position in the reference genome, select the most common aligned base from all reads to reconstruct the genome.

In [ ]:
# Step 4: Reconstruct the reference genome from consensus
reconstructed_reference = ''
for i in range(len(reference_genome)):
    if ref_alignments[i]:
        base_counts = Counter(ref_alignments[i])
        most_common_base = base_counts.most_common(1)[0][0]
        reconstructed_reference += most_common_base
    else:
        reconstructed_reference += '-'
print("\nReconstructed Reference Genome:", reconstructed_reference)
print("Reference Genome              :", reference_genome)

### Consensus Step Complexity

- **Time Complexity:** O(N × D)
  - N = reference genome length
  - D = average depth of aligned reads per position
- **Space Complexity:** O(N × D) for storing all aligned bases

The consensus is built by taking the most common base at each position.